In [1]:
from code_tokenizer import *




def math_layer(code):
    code = parase_groups(code)
    priority = {
        '+': 1,
        '-': 1,
        '*': 2,
        '/': 2,
    }
    operations = []
    variables = []
    result = []
    # create a binary tree from the operations using the priority
    for token, value in code:
        if(token == 'symbol'):
            if(value in priority.keys()):
                current_priority = priority[value]
            else:
                current_priority = 0
            while(len(operations) > 0 and current_priority <= operations[-1][0]):
                if(len(variables) > 0 and len(result) == 0):
                    result.append(variables.pop())
                if(len(variables) > 0):
                    result.append(variables.pop())
                if(len(operations) > 0):
                    result.append(operations.pop()[1])
            operations.append([current_priority,value])
        elif(token in ['word','number']):
            variables.append(value)
    busy = True
    while(busy):
        busy = False
        if(len(variables) > 0 and len(result) == 0):
            result.append(variables.pop())
            busy = True
        if(len(variables) > 0):
            result.append(variables.pop())
            busy = True
        if(len(operations) > 0):
            result.append(operations.pop()[1])
            busy = True
    return result

code = """
my_list = [1, 2, 3, 4, 5]
min = my_list[0]
for(i = 1; i < my_list.length; i++) {
    if(my_list[i] < min) {
        min = my_list[i]
    }
}
console.log(min)
"""

print(color_by_char_type(code))


# def split_code(code):
#     code = parse_groups(code)
#     cursor = [code]
#     index = [0]
#     result = []
#     while(True):
#         if(index[-1] >= len(cursor[-1])):
#             index.pop()
#             cursor.pop()
#         if(len(index) == 0):
#             break
#         def find_groups(value):
#             print(value)
#         def scan(value):
#             for i in range(len(value)):
#                 if(type(value[i]) == list):
#                     scan(value[i])
#                 else:
#                     if(value[i] in ['code_block', 'attached_block']):
#                         find_groups(value[i+1])
                    
#         find_groups(cursor[-1][index[-1]])
#         scan(cursor[-1][index[-1]])
#         # result.append(cursor[-1][index[-1]])
#         index[-1] += 1
#     return result

# for i in split_code(code):
#     print(i)
#     pass

for i in parse_groups(code):
    print(i)


my_list = [1, 2, 3, 4, 5]
min = my_list[0]
for(i = 1; i < my_list.length; i++) {
    if(my_list[i] < min) {
        min = my_list[i]
    }
}
console.log(min)

['word', 'my_list']
['symbol', '=']
['array_define', [['number', '1'], ['symbol', ','], ['number', '2'], ['symbol', ','], ['number', '3'], ['symbol', ','], ['number', '4'], ['symbol', ','], ['number', '5']]]
['word', 'min']
['symbol', '=']
['word', 'my_list', ['array_call', [['number', '0']]]]
['word', 'for', ['attached_group', [['word', 'i'], ['symbol', '='], ['number', '1'], ['symbol', ';'], ['word', 'i'], ['symbol', '<'], ['word', 'my_list'], ['symbol', '.'], ['word', 'length'], ['symbol', ';'], ['word', 'i'], ['symbol', '++']]], ['attached_block', [['word', 'if', ['attached_group', [['word', 'my_list', ['array_call', [['word', 'i']]]], ['symbol', '<'], ['word', 'min']]], ['attached_block', [['word', 'min'], ['symbol', '='], ['word', 'my_list', ['array_call', [['word', 'i']]]]]]]]]]
['word', 'console']
['symbol', '.']
['word'